In [1]:
"""Importing libraries"""
import numpy as np
import pandas as pd
import math
import os
import sys
import matplotlib.pyplot as plt

In [2]:
"""Get current working directory"""
wd = os.getcwd()

### Question 7.1a

The formula for converting __coordinates to index__ is:

$$I = x_1 + x_2 \times L1$$ 


where $L1$ = Length of individual rows, $x1$ = $x1$ coordinate, $x2$ = $x2$ coordinate, and $I$ = Index of selected coordinates.


The formula for converting __index to coordinates__ is:

$$x_1 = I\bmod L1$$
$$x_2 = \lfloor I\div L1 \rfloor$$

where $L1$ = Length of individual rows, $x1$ = $x1$ coordinate, $x2$ = $x2$ coordinate, and $I$ = Index of selected coordinates

### Question 7.1b

In [3]:
"""Setting directory of files"""
q7_wd = os.path.join(wd, "Question 7")
q7_1_wd = os.path.join(q7_wd, "Question 7.1")

In [4]:
"""Testing out the formula to convert coordinates to index"""
L1 = 4 #set number of columns
L2 = 3 #set number of rows

x1 = eval(input("Key in x1 value: ")) # I want to input some values to test out whether the formula works or not.
x2 = eval(input("Key in x2 value: "))
index = x1 + x2*L1 #formula to return x1 and x2 values
print(index)

Key in x1 value: 2
Key in x2 value: 2
10


In [63]:
"""Loading input file's coordinates"""
#read as pandas dataframe because it's more visual
input_coords = pd.read_csv(os.path.join(q7_1_wd, "input_coordinates_7_1.txt"), delimiter = "\t") 
input_coords

,x1,x2
0,16,55
1,27,10
2,48,8
3,48,2
4,5,47
5,36,48
6,24,4
7,19,55
8,43,8
9,22,38


In [79]:
"""Calculating index"""
l1 = 50 #setting length of row for calculation of index. l2 not needed for calculations on 2D plane

#Using the previously tested formula: index = x1 + x2*L1
input_coords["index"] = input_coords["x1"] + input_coords["x2"]*l1 

In [80]:
"""Creating output file"""
output_indexes = input_coords["index"]

#save output as numpy array, with header index, formatted as strings, and no comments to be made before index.
np.savetxt(os.path.join(q7_1_wd, "output_index_7_1.txt"), output_indexes.to_numpy(),header = "index", 
           fmt = "%s", comments = "") 

In [37]:
"""Testing out the formula to convert index to coordinates"""
L1 = 5
L2 = 3

index = eval(input("Key in desired index: "))
x1 = index%L1 #test formula for calculating x1
x2 = index//L1#test formula for calculating x2
print("x1 = %d; x2 = %d" % (x1,x2)) #prints out the x1 and x2 values which was generated using the formulas tested
print("coordinates =", str((x1,x2))) #formats coordinates

Key in desired index: 15
x1 = 0; x2 = 3
coordinates = (0, 3)


In [75]:
"""Calculating coordinates"""
input_index = pd.read_csv(os.path.join(q7_1_wd, "input_index_7_1.txt"))

l1 = 50 #setting length of row for calculation of index

input_index["x1"] = input_index["index"]%l1 #As per the aforementioned formula. Input into new column in dataframe
input_index["x2"] = input_index["index"]//l1 #As per the aforementioned formula. Input into new column in dataframe

In [76]:
"""Creating output file"""
output_coords = input_index[["x1", "x2"]]

np.savetxt(os.path.join(q7_1_wd, "output_coordinates_7_1.txt"), output_coords.to_numpy(),
           header = "x1\tx2", delimiter = "\t",fmt = "%s", comments = "")

### Question 7.2a

The formula for converting __coordinates to index__ in d-dimensional grid is:

$I = \sum_{i=1}^{d} \big(x_i \times \prod_{k=0}^{i-1} l_k\big)$ 

where $i$ represents the axis coordinates desired i.e. $x_i$, $d$ represents number of dimensions in total, $l_k$ represents the dimensions of specific layers selected.
_P.S. I know the part of the product function does not make sense, but it's the best way that I can put it.
The formula used in the algorithm below should be more reflective of my understanding_.


The formula for converting __index to coordinates__ in d-dimensional grid is a lot more convoluted and I had no idea how to generalise it. As such, I will just describe my thought process behind the formula that I will use in my code below. 

Basically, it is a recursive formula which always removes the values of the higher dimensions, by effect reducing the dimensions which a selected index would be involved in. An example would be for a 3x3x3x3 4D grid, if I wanted to access the x1 and x2 coordinates, I would have to first remove the possible values of  all values in a 3x3x3 3D grid which equals to 27. The modulus function is used to do so, so as to no alter any values within the 3x3x3 grid (which goes up to 26). Another way to put it would be that if i wanted to access an index which exists on the 3rd dimension, I will have to map the value which may exist on the nth dimension to the 3rd dimension. 

The coordinates within the highest dimension of d-dimensional grid will always be found using a flooring function of the index after division by the multiplication of all dimensions of all lower-dimension layers within the d-dimensional grid.

### Question 7.2b

In [12]:
"""Creating function to convert coordinates in 6-dimensional grid to index"""
l1,l2,l3,l4,l5,l6 = (4,8,5,9,6,7) #defining structure of 6D grid

# defining the products of dimensions of the various dimensions.
prod_l1=l1 
prod_l2=l1*l2
prod_l3=l1*l2*l3
prod_l4=l1*l2*l3*l4
prod_l5=l1*l2*l3*l4*l5

#defining function to generate index from the input coordinates
def to_index(x1,x2,x3,x4,x5,x6):
    index = x1 + x2*prod_l1 + x3*prod_l2 + x4*prod_l3 + x5*prod_l4 + x6*prod_l5 #formula to calculate index
    return index #return index to store in dataframe

In [244]:
"""Importing input coordinates"""
q7_wd = os.path.join(wd, "Question 7")
q7_2_wd = os.path.join(q7_wd, "Question 7.2")

#read input as dataframe because it's easier to view
input_coords = pd.read_csv(os.path.join(q7_2_wd, "input_coordinates_7_2.txt"), delimiter = "\t")
#create columns of zeroes so function can input respective values later.
input_coords["index"] = np.zeros(len(input_coords), 'int')
input_coords #test

,x1,x2,x3,x4,x5,x6,index
0,3,2,2,7,4,6,0
1,0,0,0,0,2,5,0
2,0,2,1,4,5,6,0
3,2,7,3,4,5,6,0
4,2,5,1,3,1,2,0
5,3,6,2,7,3,6,0
6,1,5,4,1,2,1,0
7,0,5,3,2,4,2,0
8,2,4,4,7,4,3,0
9,0,3,0,1,0,0,0


In [25]:
"""For every row of coordinates in dataframe, input the 6 different coordinates, and put calculated output into 
index column in dataframe"""
for row in range(len(input_coords)):
    input_coords.loc[row,"index"] = to_index(input_coords.loc[row,"x1"],
                                            input_coords.loc[row,"x2"],
                                            input_coords.loc[row,"x3"],
                                            input_coords.loc[row,"x4"],
                                            input_coords.loc[row,"x5"],
                                            input_coords.loc[row,"x6"])

In [40]:
"""Creating output file"""
output_indexes = input_coords["index"]
np.savetxt(os.path.join(q7_2_wd, "output_index_7_2.txt"), output_indexes.to_numpy(),header = "index", 
           fmt = "%s", comments = "")

In [41]:
"""Importing input index"""
input_index = pd.read_csv(os.path.join(q7_2_wd, "input_index_7_2.txt"), delimiter = "\t")

In [ ]:
"""Initialising important variables"""
l1,l2,l3,l4,l5,l6 = (4,8,5,9,6,7) #defining structure of 6D grid

# defining the products of dimensions of the various dimensions.
prod_l1=l1
prod_l2=l1*l2
prod_l3=l1*l2*l3
prod_l4=l1*l2*l3*l4
prod_l5=l1*l2*l3*l4*l5

In [45]:
"""Calculation of each coordinates after mapping back index to the correct dimension"""
input_index["x1"] = input_index["index"]%prod_l5%prod_l4%prod_l3%prod_l2%prod_l1
input_index["x2"] = input_index["index"]%prod_l5%prod_l4%prod_l3%prod_l2//prod_l1
input_index["x3"] = input_index["index"]%prod_l5%prod_l4%prod_l3//prod_l2
input_index["x4"] = input_index["index"]%prod_l5%prod_l4//prod_l3
input_index["x5"] = input_index["index"]%prod_l5//prod_l4
input_index["x6"] = input_index["index"]//prod_l5

In [47]:
"""Creating output file"""
output_coords = input_index[["x1", "x2", "x3", "x4", "x5", "x6"]]

#Convert table to numpy, save with the respective column headers, delimited by tabs in string format.
np.savetxt(os.path.join(q7_2_wd, "output_coordinates_7_2.txt"), output_coords.to_numpy(),
           header = "x1\tx2\tx3\tx4\tx5\tx6", delimiter = "\t",fmt = "%s", comments = "")